In [1]:
import os
import subprocess
from transformers import AutoModelForCausalLM, AutoTokenizer
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters
import nest_asyncio
import asyncio

# Apply `nest_asyncio` to fix nested event loops in Jupyter/Colab
nest_asyncio.apply()

# Install dependencies
print("Installing dependencies. This might take a few minutes...")
dependencies = ["transformers", "torch", "python-telegram-bot"]
for dependency in dependencies:
    subprocess.run(["pip", "install", dependency], check=True)

# LLM setup
MODEL_NAME = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
print(f"Loading model {MODEL_NAME}. This might take a while...")
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
print("Model loaded successfully.")

# Telegram bot setup
TELEGRAM_API_TOKEN = ""  # Replace with your bot's API token

if TELEGRAM_API_TOKEN == "YOUR_TELEGRAM_BOT_API_TOKEN":
    raise ValueError("Please replace 'YOUR_TELEGRAM_BOT_API_TOKEN' with your actual API token from BotFather.")

def generate_response(input_text):
    """Generate a response using the LLM."""
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=50)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

async def handle_message(update, context):
    """Handle incoming messages from Telegram users."""
    user_message = update.message.text.lower()
    print(f"Received message: {user_message}")

    # Dictionary to store predefined animal-related questions and answers
    animal_facts = {
        "tell me about dogs": (
            "Dogs are domesticated animals that have been companions to humans for thousands of years. "
            "They are known for their loyalty, intelligence, and unique bond with humans."
        ),
        "what is the history of dogs": (
            "Dogs were domesticated from wolves over 15,000 years ago. They were initially bred for hunting, guarding, and companionship. "
            "Today, there are hundreds of breeds with diverse traits and purposes."
        ),
        "how do dogs help humans": (
            "Dogs assist humans in many ways: as service animals for people with disabilities, in search-and-rescue missions, herding livestock, "
            "and even providing emotional support and companionship."
        ),
        "why are dogs called man's best friend": (
            "Dogs are called man's best friend because of their unwavering loyalty, companionship, and the strong emotional bond they share with humans. "
            "Their ability to understand and respond to human emotions further strengthens this relationship."
        ),
        "how do dogs communicate": (
            "Dogs communicate through body language, barking, whining, growling, and tail wagging. They also interpret human gestures, facial expressions, "
            "and tones of voice to understand us better."
        ),
        "tell me about cats": (
            "Cats are small, carnivorous mammals that have been domesticated for thousands of years. They are known for their independence, agility, and playful nature."
        ),
        "how do cats communicate": (
            "Cats communicate using body language, vocalizations like meowing and purring, and even subtle behaviors like blinking slowly at their owners."
        ),
        "tell me about elephants": (
            "Elephants are the largest land animals on Earth, known for their intelligence, strong social bonds, and memory. They are found in Africa and Asia."
        ),
        "how do elephants communicate": (
            "Elephants communicate through a variety of sounds, including trumpeting and low-frequency rumbles, as well as through touch and body language."
        ),
        "tell me about birds": (
            "Birds are warm-blooded vertebrates characterized by feathers, beaks, and the ability to lay eggs. They are found in diverse environments worldwide."
        ),
        "how do birds communicate": (
            "Birds communicate using songs, calls, and visual displays like plumage patterns or dances, especially during mating rituals."
        ),
        "yes": (
            "Thank you for your confirmation. If you have more questions or need additional information, feel free to ask!"
        )
    }

    # Generate a response based on predefined animal facts
    response = animal_facts.get(user_message, None)

    # If the user's message doesn't match predefined facts, use the model to generate a response
    if response is None:
        response = generate_response(user_message)

    print(f"Generated response: {response}")
    await update.message.reply_text(response)

async def start(update, context):
    """Handle the /start command."""
    await update.message.reply_text("Hello! I'm your AI assistant. Ask me anything about animals!")

async def main():
    """Main function to run the Telegram bot."""
    application = ApplicationBuilder().token(TELEGRAM_API_TOKEN).build()

    application.add_handler(CommandHandler("start", start))
    application.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, handle_message))

    print("Starting the bot...")
    await application.run_polling()

if __name__ == "__main__":
    # Run the bot using asyncio to handle the event loop
    asyncio.run(main())


Installing dependencies. This might take a few minutes...
Loading model TinyLlama/TinyLlama-1.1B-Chat-v1.0. This might take a while...


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Model loaded successfully.
Starting the bot...
Received message: tell me about dogs
Generated response: Dogs are domesticated animals that have been companions to humans for thousands of years. They are known for their loyalty, intelligence, and unique bond with humans.
Received message: what is the history of dogs
Generated response: Dogs were domesticated from wolves over 15,000 years ago. They were initially bred for hunting, guarding, and companionship. Today, there are hundreds of breeds with diverse traits and purposes.
Received message: how do dogs help humans
Generated response: Dogs assist humans in many ways: as service animals for people with disabilities, in search-and-rescue missions, herding livestock, and even providing emotional support and companionship.
Received message: why are dogs called man's best friend
Generated response: Dogs are called man's best friend because of their unwavering loyalty, companionship, and the strong emotional bond they share with humans. Th

RuntimeError: Cannot close a running event loop